In [53]:
import pymaid
url = 'https://neurophyla.mrc-lmb.cam.ac.uk/catmaid/drosophila/l1/seymour/#'
token = "x"
name = "x"
password = "x"
project_id = 1
rm = pymaid.CatmaidInstance(url, token, name, password, project_id)

INFO  : Global CATMAID instance set. Caching is ON. (pymaid)


# 1.0 Clarify Df 

In [4]:
import pandas as pd 
df = pd.read_csv('~/Documents/CATMAID/pymaid/10_neurotransmittersSeymour/202502-starting_larval_gt_data.csv')
df

,cell_type,known_nt,known_nt_source,known_nt_confidence,species,region,hemilineage
0,13a ORN left,acetylcholine,"Winding, Pedigo et al. 2023, Hueckesfeld et al...",4,larval_drosophila_melanogaster,cns,unpublished
1,13a ORN right,acetylcholine,"Winding, Pedigo et al. 2023, Hueckesfeld et al...",4,larval_drosophila_melanogaster,cns,unpublished
2,13a PN left,acetylcholine,"Winding, Pedigo et al. 2023, Miroschnikow et a...",4,larval_drosophila_melanogaster,cns,unpublished
3,13a PN right,acetylcholine,"Miroschnikow et al. 2018, Winding, Pedigo et a...",4,larval_drosophila_melanogaster,cns,unpublished
4,1a ORN left,acetylcholine,"Winding, Pedigo et al. 2023, Hueckesfeld et al...",4,larval_drosophila_melanogaster,cns,unpublished
...,...,...,...,...,...,...,...
799,vbd_a1r,acetylcholine,Heckscher et al. 2015,2,larval_drosophila_melanogaster,cns,unpublished
800,vchA/B a1l,acetylcholine,"Barnes et al. 2022, Jovanic et al. 2019, Valde...",4,larval_drosophila_melanogaster,cns,unpublished
801,vchA/B a1r,acetylcholine,"Barnes et al. 2022, Jovanic et al. 2019, Valde...",4,larval_drosophila_melanogaster,cns,unpublished
802,vpda_t3l,acetylcholine,Heckscher et al. 2015,2,larval_drosophila_melanogaster,cns,unpublished


In [7]:
df['known_nt'].unique()

array(['acetylcholine', 'glutamate', 'gaba', 'serotonin', 'octopamine',
       'dopamine'], dtype=object)

In [30]:
neurons=df['cell_type'].tolist()

# 2.0 From cell type, get skids from CATMAID

In [34]:
skids = pymaid.get_skids_by_name(neurons)
n = pymaid.get_neurons(skids, with_connectors=True)
n

Fetching names:   0%|          | 0/796 [00:00<?, ?it/s]

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


Fetch neurons:   0%|          | 0/963 [00:00<?, ?it/s]

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


Make nrn:   0%|          | 0/963 [00:00<?, ?it/s]

,type,name,skeleton_id,n_nodes,n_connectors,n_branches,n_leafs,cable_length,soma,units
0,CatmaidNeuron,13a ORN left,7527710,894,130,39,41,104724.382812,None,1 nanometer
1,CatmaidNeuron,13a ORN right,4073353,1276,164,55,63,134808.968750,None,1 nanometer
...,...,...,...,...,...,...,...,...,...,...
961,CatmaidNeuron,vpda_t3l,11578922,919,77,38,43,140493.984375,None,1 nanometer
962,CatmaidNeuron,vpda_t3r,8234536,954,91,41,46,148984.468750,None,1 nanometer


In [40]:
pymaid = n.summary() 
merged_df = pd.merge(df, pymaid, left_on='cell_type', right_on='name', how='inner')
columns = ['cell_type', 'skeleton_id','known_nt', 'n_connectors', 'region']
revised = merged_df[columns]
print(revised.head())

       cell_type skeleton_id       known_nt  n_connectors region
0   13a ORN left     7527710  acetylcholine           130    cns
1  13a ORN right     4073353  acetylcholine           164    cns
2    13a PN left     4493199  acetylcholine           204    cns
3   13a PN right     8244823  acetylcholine           255    cns
4    1a ORN left     2611805  acetylcholine            96    cns


In [42]:
def get_unique_neurons_by_neurotransmitter(df, neurotransmitter):
    """
    Returns a list of unique neuron cell types that use the specified neurotransmitter.

    Parameters:
        df (pd.DataFrame): The DataFrame containing the neuron data.
        neurotransmitter (str): The neurotransmitter to filter

    Returns:
        List of unique neuron cell types using the given neurotransmitter.
    """
    filtered_df = df[df["known_nt"] == neurotransmitter]
    return filtered_df["skeleton_id"].unique().tolist()

In [49]:
nt_neurons = get_unique_neurons_by_neurotransmitter(revised, "serotonin")
print(nt_neurons)

['13849557', '6143930', '7982896', '8177199']


# 2.0 Find post_partners from selected neurons 

In [57]:
partner = pymaid.get_partners(nt_neurons, directions=['outgoing'], remote_instance=None)

INFO  : Fetching connectivity table for 4 neurons (pymaid)
INFO  : Done. Found 0 pre-, 528 postsynaptic and 0 gap junction-connected neurons (pymaid)


In [58]:
partner

,neuron_name,skeleton_id,num_nodes,relation,13849557,6143930,7982896,8177199,total
0,keystone left,5030808,8286,downstream,16,24,0,0,40
1,keystone right,10831005,7369,downstream,13,20,0,0,33
2,mPN iACT C2 left,2612527,4995,downstream,0,23,0,0,23
3,mPN iACT C2 right,4391978,6347,downstream,22,0,0,0,22
4,mPN iACT B1_right,10167078,4763,downstream,14,1,0,0,15
...,...,...,...,...,...,...,...,...,...
523,33a PN right,13589251,3288,downstream,1,0,0,0,1
524,AF1 left,9408003,2952,downstream,1,0,0,0,1
525,bilateral R,6597872,3927,downstream,1,0,0,0,1
526,neuron 10002122,10002121,39,downstream,1,0,0,0,1


In [59]:
post=partner['skeleton_id'].tolist()

# 3.0 Get pre and post info 
Postsynaptic partners determined from above are used as a target group for function `get_connectors_between`. This ensures feature of synapses are retrieved from the neuron of interest. 

In [86]:
links= pymaid.get_connectors_between(nt_neurons, post, directional=True, remote_instance=None)

INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)
INFO  : Cached data used. Use `pymaid.clear_cache()` to clear. (pymaid)


In [87]:
links

,connector_id,connector_loc,node1_id,source_neuron,confidence1,creator1,node1_loc,node2_id,target_neuron,confidence2,creator2,node2_loc
0,14250706,"[73815.0, 22621.4, 59100.0]",6158808,6143930,5,claus,"[73606.0, 22708.8, 59100.0]",12217195,2379529,5,claus,"[74020.2, 22450.4, 59250.0]"
1,14250706,"[73815.0, 22621.4, 59100.0]",6158808,6143930,5,claus,"[73606.0, 22708.8, 59100.0]",7220937,7851153,5,claus,"[73482.5, 22252.8, 59100.0]"
2,14250706,"[73815.0, 22621.4, 59100.0]",6158808,6143930,5,claus,"[73606.0, 22708.8, 59100.0]",14250731,15734988,5,claus,"[74164.6, 22363.0, 59200.0]"
3,6898319,"[80434.6, 62726.6, 41050.0]",6150692,6143930,5,matt,"[80512.5, 62529.0, 41050.0]",6791893,7941642,5,matt,"[80480.2, 62903.3, 41050.0]"
4,6898319,"[80434.6, 62726.6, 41050.0]",6150692,6143930,5,matt,"[80512.5, 62529.0, 41050.0]",6898326,6791927,5,matt,"[80290.2, 62802.6, 41050.0]"
...,...,...,...,...,...,...,...,...,...,...,...,...
1252,24091340,"[41682.2, 20971.25, 35100.0]",5556182,13849557,5,akira,"[42135.35, 20874.35, 35100.0]",9503809,3765728,5,akira,"[41465.6, 20888.6, 35100.0]"
1253,24091340,"[41682.2, 20971.25, 35100.0]",5556182,13849557,5,akira,"[42135.35, 20874.35, 35100.0]",23331202,10002121,5,akira,"[41744.1, 21066.05, 35050.0]"
1254,24732252,"[71814.05, 34832.2, 58100.0]",5559878,13849557,5,akira,"[72283.6, 34895.4, 58100.0]",22892006,9409359,5,akira,"[71612.01, 34792.566, 58100.0]"
1255,24975997,"[31848.316, 18009.95, 45800.0]",5563763,13849557,5,sebastian,"[31973.2, 17871.4, 45800.0]",9964362,4983938,5,sebastian,"[31762.3, 18150.7, 45800.0]"


In [90]:
links['source_neuron'].unique()

array([ 6143930,  7982896,  8177199, 13849557])

In [96]:
df = links.copy()
df[['connector_x', 'connector_y', 'connector_z']] = pd.DataFrame(df['connector_loc'].tolist(), index=df.index)
df[['pre_x', 'pre_y', 'pre_z']] = pd.DataFrame(df['node1_loc'].tolist(), index=df.index)
df[['post_x', 'post_y', 'post_z']] = pd.DataFrame(df['node2_loc'].tolist(), index=df.index)
df.rename(columns={
    'node1_id': 'prenode_id',       
    'source_neuron': 'pre_neuron',
    'node2_id': 'postnode_id',
    'target_neuron': 'post_neuron'
}, inplace=True)
df = df.drop(columns=['creator1','creator2','confidence1','confidence2','connector_loc','node1_loc','node2_loc'])
df.to_csv('serotonin_neurons.csv')
df.head()

,connector_id,prenode_id,pre_neuron,postnode_id,post_neuron,connector_x,connector_y,connector_z,pre_x,pre_y,pre_z,post_x,post_y,post_z
0,14250706,6158808,6143930,12217195,2379529,73815.0,22621.4,59100.0,73606.0,22708.8,59100.0,74020.2,22450.4,59250.0
1,14250706,6158808,6143930,7220937,7851153,73815.0,22621.4,59100.0,73606.0,22708.8,59100.0,73482.5,22252.8,59100.0
2,14250706,6158808,6143930,14250731,15734988,73815.0,22621.4,59100.0,73606.0,22708.8,59100.0,74164.6,22363.0,59200.0
3,6898319,6150692,6143930,6791893,7941642,80434.6,62726.6,41050.0,80512.5,62529.0,41050.0,80480.2,62903.3,41050.0
4,6898319,6150692,6143930,6898326,6791927,80434.6,62726.6,41050.0,80512.5,62529.0,41050.0,80290.2,62802.6,41050.0


In [123]:
import pandas as pd

centroid = df.copy()
neurotransmitter = 'serotonin'
centroid.insert(0, 'classification', neurotransmitter)

# Step 1: Convert coordinates from nanometers to voxel units (pixels)
def convert_to_pixels(df):
    for axis in ['x', 'y']:
        for prefix in ['connector_', 'pre_', 'post_']:
            df[f'{prefix}{axis}'] = df[f'{prefix}{axis}'] / 4 

    for prefix in ['connector_', 'pre_', 'post_']:
        df[f'{prefix}z'] = df[f'{prefix}z'] / 50  # 50 nm per voxel in z

    return df

centroid = convert_to_pixels(centroid)

def calculate_bbox_and_check_from_connector(row, x_exp, y_exp, z_exp):
    # New center: the synaptic connector location
    cx = row['connector_x']
    cy = row['connector_y']
    cz = row['connector_z']

    # Bounding box range
    xmin, xmax = cx - x_exp, cx + x_exp
    ymin, ymax = cy - y_exp, cy + y_exp
    zmin, zmax = cz - z_exp, cz + z_exp

    # Check if pre and post coords are within the bounding box
    x_out = not (xmin <= row['pre_x'] <= xmax and xmin <= row['post_x'] <= xmax)
    y_out = not (ymin <= row['pre_y'] <= ymax and ymin <= row['post_y'] <= ymax)
    z_out = not (zmin <= row['pre_z'] <= zmax and zmin <= row['post_z'] <= zmax)

    return pd.Series({
        'centroid_x': cx, 'centroid_y': cy, 'centroid_z': cz,
        'bbox_xmin': xmin, 'bbox_xmax': xmax,
        'bbox_ymin': ymin, 'bbox_ymax': ymax,
        'bbox_zmin': zmin, 'bbox_zmax': zmax,
        'any_out_of_range': x_out or y_out or z_out
    })


expansion_x_px = 80
expansion_y_px = 80
expansion_z_px = 50  

centroid[['centroid_x', 'centroid_y', 'centroid_z',
          'bbox_xmin', 'bbox_xmax', 'bbox_ymin', 'bbox_ymax',
          'bbox_zmin', 'bbox_zmax', 'any_out_of_range']] = centroid.apply(
    calculate_bbox_and_check_from_connector,
    axis=1,
    args=(expansion_x_px, expansion_y_px, expansion_z_px)
)

# Columns to save
columns_to_save = ['classification','connector_id','prenode_id','pre_neuron','postnode_id','post_neuron','connector_x','connector_y','connector_z',
                   'pre_x','pre_y','pre_z','post_x','post_y','post_z','bbox_xmin', 'bbox_xmax', 'bbox_ymin', 'bbox_ymax', 'bbox_zmin', 'bbox_zmax','any_out_of_range']


out_of_range_filename = f'out_of_range_{neurotransmitter}.csv'
bbox_filename = f'{neurotransmitter}_bbox.csv'

out_of_range_data = centroid[centroid['any_out_of_range']]
if not out_of_range_data.empty:
    out_of_range_data[columns_to_save].to_csv(out_of_range_filename, index=False)
    print(f"Out of range data saved to '{out_of_range_filename}'.")
else:
    print("No out of range data found.")


centroid[columns_to_save].to_csv(bbox_filename, index=False)
print(f"Full data with bounding boxes saved to '{bbox_filename}'.")

Out of range data saved to 'out_of_range_serotonin.csv'.
Full data with bounding boxes saved to 'serotonin_bbox.csv'.


In [119]:
import numpy as np

dx = (centroid['post_x'] - centroid['connector_x']).abs()
dy = (centroid['post_y'] - centroid['connector_y']).abs()
dz = (centroid['post_z'] - centroid['connector_z']).abs()

mean_dx_px = dx.mean()
mean_dy_px = dy.mean()
mean_dz_px = dz.mean()

p90_dx_px = np.percentile(dx, 90)
p90_dy_px = np.percentile(dy, 90)
p90_dz_px = np.percentile(dz, 90)

print(" Axis-wise average distances (pixels):")
print(f"X: {mean_dx_px:.2f} px,  90th percentile: {p90_dx_px:.2f} px")
print(f"Y: {mean_dy_px:.2f} px,  90th percentile: {p90_dy_px:.2f} px")
print(f"Z: {mean_dz_px:.2f} px,  90th percentile: {p90_dz_px:.2f} px")


📏 Axis-wise average distances (pixels):
X: 49.81 px,  90th percentile: 95.52 px
Y: 48.70 px,  90th percentile: 95.00 px
Z: 0.61 px,  90th percentile: 2.00 px
